## Sage Data Merge

In [39]:
import os, pandas as pd, numpy as np, re
from functools import reduce
pd.set_option('display.max_columns', 100)

#NOTE TO SELF: redo the relative paths.
sage = '/home/wraikes/Dropbox/partnership/DMTBilly data - Copy/Sage Data'
os.chdir(sage)

In [40]:
#GLOBAL VARIABLE
new_dfs = {}

#Functions for data processing
def remove_dupes(df):
    
    test_users = ['ThpMV2Achc', 'SEkQVTCe6j', 'Wh8NSX3DHL', 'SaXFr2kPZa', 'VWUcSp4TeH', 'yXEfAmW682', 
                  'gwEpQR8j9B', 'WbbNWM4RAF', 'D5bzYrfd8E', 'LJcmEFWp74', 'ULoF3MM1nN', 'bdyP3M']
    
    return df[~df.externalId.isin(test_users)]

def dupe_check(df):
    return len(df.externalId) == len(df.externalId.unique()) and len(df.externalId) > 0

def new_cols(df, append):
    # Remove: 'metadata.json.' and 'data.json.'.
    col_re = re.compile('metadata.json.|data.json.')
    df.columns = list(map(lambda x: re.sub(col_re, '', x), df.columns))
    
    # Append df identifier to columns.
    df.columns = list(map(lambda x: append + x, df.columns))
    df = df.rename(columns = {append + 'externalId': 'externalId'})

    return df

def create_new_df(df, att, var_1, var_2=None):
    cols = df.columns
    
    new_df = pd.DataFrame(columns = cols)
        
    for ix, row in df.iterrows():
        if var_2:
            if row[var_1] == att[1] and row[var_2] == att[2]:
                new_df = new_df.append(row, ignore_index=True)
        else:
            if row[var_1] == att[1]:
                new_df = new_df.append(row, ignore_index=True)
    
    return new_df

def df_merge(df, attributes, var_1, var_2=None):
    dfs = []
    
    for att in attributes:
        new_df = create_new_df(df, att, var_1, var_2)
        new_df = new_cols(new_df, att[0])
        dfs.append(new_df)
    
    df_merge = reduce(lambda left, right: pd.merge(left, 
                                                   right, 
                                                   how = 'outer',
                                                   on='externalId'),
                      dfs)
    
    return df_merge

def final_check(df, name):
    if dupe_check(df):
        new_dfs[name] = df
        print('Done!')
    else:
        print('False')

### Process files that do not need cleaning / restructuring.

In [41]:
test_users = ['ThpMV2Achc', 'SEkQVTCe6j', 'Wh8NSX3DHL', 'SaXFr2kPZa', 'VWUcSp4TeH', 'yXEfAmW682', 
              'gwEpQR8j9B', 'WbbNWM4RAF', 'D5bzYrfd8E', 'LJcmEFWp74', 'ULoF3MM1nN', 'bdyP3M']

files_to_exclude = ['digital-marshmallow-status_8.8.17.csv',
                    'digital-marshmallow-appVersion_8.8.17.csv']

files_as_is = []

for file in os.listdir():
    if file not in files_to_exclude:
        df = pd.read_csv(file)
        df = remove_dupes(df)
        if dupe_check(df):
            files_as_is.append(file)
        else:
            continue

In [42]:
len(files_as_is)

10

In [43]:
files_as_is

['digital-marshmallow-past_year_likert_21-v2_8.8.17.csv',
 'digital-marshmallow-behavior_choices_1_bl-v1_8.8.17.csv',
 'digital-marshmallow-comments_21-v2_8.8.17.csv',
 'digital-marshmallow-demographics-v2_8.8.17.csv',
 'digital-marshmallow-behavior_likert_21-v2_8.8.17.csv',
 'digital-marshmallow-behavior_likert_bl-v2_8.8.17.csv',
 'digital-marshmallow-generally_sem_diff_bl-v2_8.8.17.csv',
 'digital-marshmallow-generally_sem_diff_21-v2_8.8.17.csv',
 'digital-marshmallow-as_a_child_likert_bl-v2_8.8.17.csv',
 'digital-marshmallow-past_year_likert_bl-v3_8.8.17.csv']

In [5]:
names_as_is = ['SAGE_past_year_likert_21_-_', 
               'SAGE_behavior_choices_1_-_',
               'SAGE_comments_21_-_',
               'SAGE_demos_-_',
               'SAGE_behavior_lk_21_-_',
               'SAGE_behavior_lk_bl_-_',
               'SAGE_generally_sem_bl_-_',
               'SAGE_generally_sem_21_-_',
               'SAGE_as_a_child_-_',
               'SAGe_past_year_likert_bl_-_']

In [6]:
for file, name in zip(files_as_is, names_as_is):
    df = pd.read_csv(file)
    df = remove_dupes(df)
    df = new_cols(df, name)
    new_dfs[name] = df

In [7]:
all([dupe_check(df) for name, df in new_dfs.items()])

True

### DataFrame: Bart_V4

In [8]:
bart_v4 = pd.read_csv('digital-marshmallow-bart-v4_8.8.17.csv')
bart_v4 = remove_dupes(bart_v4)

dupe_check(bart_v4)

False

In [9]:
bart_attributes = [
    ['SAGE_bart_bl_0.25_-_', 'baseline', 'BART0.25'],
    ['SAGE_bart_bl_250_-_', 'baseline', 'BART250.00'],
    ['SAGE_bart_21_0.25_-_', '21-day-assessment', 'BART250.00'],
    ['SAGE_bart_21_250_-_', '21-day-assessment', 'BART250.00']
]

In [10]:
bart_v4 = df_merge(bart_v4, 
                   bart_attributes,
                   var_1='metadata.json.taskIdentifier',
                   var_2='data.json.variable_label')

In [11]:
final_check(bart_v4, 'bart_v4')

Done!


### DataFrame: Behavior_choices_4

In [12]:
behavior_4 = pd.read_csv('digital-marshmallow-behavior_choices_4_bl-v2_8.8.17.csv')
behavior_4 = remove_dupes(behavior_4)

dupe_check(behavior_4)

False

In [13]:
behavior_4_attributes = [
    ['SAGE_behavior_4_bl_-_', 'baseline'],
]

In [14]:
behavior_4 = df_merge(behavior_4, 
                      behavior_4_attributes,
                      var_1='metadata.json.taskIdentifier')

In [15]:
final_check(behavior_4, 'behave_4')

Done!


### DataFrame: Delay Discounting

In [16]:
delay = pd.read_csv('digital-marshmallow-delay_discounting_raw-v6_8.8.17.csv')
delay = remove_dupes(delay)

dupe_check(delay)

False

In [17]:
bl = 'baseline'
_21 = '21-day-assessment'

delay_attributes = [
    ['SAGE_delay_bl_time_6_month_-_', bl, 'dd_time_6_month'],
    ['SAGE_delay_bl_money_6_month_-_', bl, 'dd_money_6_month'],
    ['SAGE_delay_bl_money_1_month_-_', bl, 'dd_money_1_month'],
    ['SAGE_delay_bl_time_1_year_-_', bl, 'dd_time_1_year'],
    ['SAGE_delay_21_time_6_month_-_', _21, 'dd_time_6_month'],
    ['SAGE_delay_21_money_6_month_-_', _21, 'dd_money_6_month'],
    ['SAGE_delay_21_money_1_month_-_', _21, 'dd_money_1_month'],
    ['SAGE_delay_21_time_1_year_-_', _21, 'dd_time_1_year']
]

In [18]:
delay = df_merge(delay, 
                 delay_attributes, 
                 var_1='metadata.json.taskIdentifier',
                 var_2='data.json.variableLabel')

In [19]:
final_check(delay, 'delay')

Done!


### DataFrame: Discounting Raw

In [20]:
discount = pd.read_csv('digital-marshmallow-discounting_raw-v2_8.8.17.csv')
discount = remove_dupes(discount)

dupe_check(discount)

False

In [21]:
bl = 'baseline'
_21 = '21-day-assessment'

discount_attributes = [
    ['SAGE_discount_bl_money_-_', bl, 'pd_constant_money'],
    ['SAGE_discount_bl_prob_-_', bl, 'pd_constant_probabiliy'],
    ['SAGE_discount_21_money_-_', _21, 'pd_constant_money'],
    ['SAGE_discount_21_prob_-_', _21, 'pd_constant_probability']
]

discount = df_merge(discount, 
                    discount_attributes,
                    var_1='metadata.json.taskIdentifier',
                    var_2='data.json.variableLabel')

In [22]:
final_check(discount, 'discount')

Done!


### DataFrame: Evening Notification

In [23]:
evening_note = pd.read_csv('digital-marshmallow-evening_notification_time-v2_8.8.17.csv')
evening_note = remove_dupes(evening_note)

dupe_check(evening_note)

False

In [24]:
evening_note_attributes = [
    ['SAGE_evening_note_bl_-_', 'baseline']
]

evening_note_bl = df_merge(evening_note, 
                           evening_note_attributes,
                           var_1='metadata.json.taskIdentifier')

In [25]:
final_check(evening_note_bl, 'evening_note')

Done!


### DataFrame: GoNoGo - PLACEHOLDER (Extra Record - ksJM3Y)

In [26]:
gonogo = pd.read_csv('digital-marshmallow-goNoGo-v2_8.8.17.csv')
gonogo = remove_dupes(gonogo)

dupe_check(gonogo)

False

In [27]:
bl = 'baseline'
_21 = '21-day-assessment'

gonogo_attributes = [
    ['SAGE_gonogo_bl_stable_-_', bl, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', bl, 'go_no_go_variable_stimulus_active_task'],
    ['SAGE_gonogo_bl_stable_-_', _21, 'go_no_go_stable_stimulus_active_task'],
    ['SAGE_gonogo_21_variable_-_', _21, 'go_no_go_variable_stimulus_active_task']
]

gonogo = df_merge(gonogo, 
                  gonogo_attributes,
                  var_1='metadata.json.taskIdentifier',
                  var_2='data.json.variable_label')

In [28]:
final_check(gonogo, 'gonogo')

False


In [29]:
gonogo.externalId[gonogo.externalId.duplicated()]

13    ksJM3Y
Name: externalId, dtype: object

### DataFrame: Morning Notifications

In [30]:
morning_note = pd.read_csv('digital-marshmallow-morning_notification_time-v3_8.8.17.csv')
morning_note = remove_dupes(morning_note)

dupe_check(morning_note)

False

In [31]:
morning_note_attributes = [
    ['SAGE_morning_note_bl_-_', 'baseline']
]

morning_note = df_merge(morning_note, 
                        morning_note_attributes,
                        var_1='metadata.json.taskIdentifier')

In [32]:
final_check(morning_note, 'morning_note')

Done!


### DataFrame: PAM Multiple

In [33]:
pam_mult = pd.read_csv('digital-marshmallow-pam_multiple-v2_8.8.17.csv')
pam_mult = remove_dupes(pam_mult)

dupe_check(pam_mult)

False

In [34]:
pam_mult_attributes = [
    ['SAGE_pam_mult_bl_-_', 'baseline'],
    ['SAGE_pam_mult_21_-_', '21-day-assessment']
]

pam_mult = df_merge(pam_mult, 
                    pam_mult_attributes,
                    var_1='metadata.json.taskIdentifier')

In [35]:
final_check(pam_mult, 'pam_mult')

Done!


### Final Merge of All Sage Data

In [36]:
final_df = reduce(lambda left, right: pd.merge(left, right, how = 'outer',
                                               on='externalId'), 
                  new_dfs.values())

In [37]:
dupe_check(final_df)

True

In [38]:
os.chdir('/home/wraikes/Programming/Partnership/dmt/merged_data/')
final_df.to_csv('FINAL_SAGE.csv')